### Forecasting all popular girls names

In [1]:
#%pip install pmdarima

     |████████████████████████████████| 1.4 MB 4.4 MB/s 
     |████████████████████████████████| 9.8 MB 22.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
import pandas as pd
import numpy as np
import statsmodels
import pmdarima as pmd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime
from statsmodels.tsa.api import SimpleExpSmoothing, ExponentialSmoothing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN, LSTM, GRU, Dense
import datetime

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
female_pop=pd.read_csv('/content/drive/MyDrive/Data/popular_females.csv', parse_dates =['year'], index_col="year")
female_names = pd.read_csv("/content/drive/MyDrive/Data/female_names.csv",  parse_dates =['year'], index_col="year")
names_100_years = pd.read_csv("/content/drive/MyDrive/Data/females_100count_10years_1920.csv", parse_dates =['year'], index_col="year")

### Pull all popular girl names in a list/size it

In [9]:
females_name_list = female_pop["name"].unique().tolist()

In [10]:
len(females_name_list)

1058

### RUN ARIMA and PREDICTION MODEL ON FULL DATASET (1880-2020)
and Predict for names out to 2036 (So for 15 years)

In [11]:

def name_prediction(names, df, sex):
    one_big_list =[]
    for name in names:
        
    
        Z = df[df["name"]==name]

        train = Z[['count']]
        index = pd.date_range(start="01/01/2021", end="01/01/2037", freq="y")
        test=pd.DataFrame(index=index)
        model = pmd.auto_arima(train["count"], error_action='ignore')
        prediction = model.predict(len(test))
        test["count"] = np.where(prediction>0, prediction, 0)
        test["name"] = name
        test["sex"] = sex
        final_data = Z.append(test, ignore_index=False)
        one_big_list.append(final_data)
    return pd.concat(one_big_list, axis=0)

In [12]:
final_data_females = name_prediction(females_name_list, female_names, "F")

In [13]:
final_data_females.drop(columns=["rank"], inplace=True)

In [14]:
final_data_females.to_csv('/content/drive/MyDrive/Data/final_data_females.csv')

In [15]:
final_data_females.head()

,name,sex,count
1880-01-01,Amelia,F,221.0
1881-01-01,Amelia,F,235.0
1882-01-01,Amelia,F,252.0
1883-01-01,Amelia,F,262.0
1884-01-01,Amelia,F,315.0


In [16]:
final_data_females.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 121607 entries, 1880-01-01 to 2036-12-31
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   name    121607 non-null  object 
 1   sex     121607 non-null  object 
 2   count   121607 non-null  float64
dtypes: float64(1), object(2)
memory usage: 3.7+ MB


In [17]:
import pickle

In [18]:
final_data_females.to_pickle('/content/drive/MyDrive/Data/final_data_females.pkl')